# User Setup

In [21]:

import json
import base64
import requests
import pandas as pd



################# this is web testing ##############################
#client_id = '997B883FBD8F4100B021ABB5D83C5DE3' # web client id
#client_secret = 'Bk90_tGJsnRY6LrkvzneoXPLajzU04Mq4EW0ub5-iS7G2UBV'
#client_secret = 'rsKvrpFBQ4p_lCXKLF3JtB7VUUQTHJElckpvw8mt49dVMkSx'
####################################################################


############### Reza API #########################################
client_id = 'B628C81B8827482AAC0E3E7BA73676F2'
client_secret = '890a2f6bbcf77d0dcd68769068fa646590d4005d09c3e1a3685d7f0e1194e8e2'
###################################################################################

redirect_url = 'https://developer.xero.com/'
#scope = 'offline_access accounting.contacts'
scope = 'offline_access accounting.reports.read'
b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')

In [2]:
auth_url = ('https://login.xero.com/identity/connect/authorize?' +
'response_type=code' +
'&client_id=' + client_id +
'&scope=' + scope +
'&redirect_uri=' + redirect_url)
print (auth_url)

https://login.xero.com/identity/connect/authorize?response_type=code&client_id=997B883FBD8F4100B021ABB5D83C5DE3&scope=offline_access accounting.reports.read&redirect_uri=https://developer.xero.com/


In [22]:
code = '5c7eeb3c9f8578ea608cbe42cbc93353052d803a858d1e0495769d8289ccda73'

In [23]:
 # 3. Exchange the code
exchange_code_url = 'https://identity.xero.com/connect/token'
response = requests.post(exchange_code_url, 
                        headers = {
                            'Content-Type': 'application/x-www-form-urlencoded'
                        },
                        data={
              'grant_type': 'authorization_code',
              'code': code,
              'redirect_uri': redirect_url
})


# save refresh token
f = open('oath_refresh2.txt','w')
refresh_token = response.json()['refresh_token']
f.write(refresh_token)


KeyError: 'refresh_token'

In [24]:
response

<Response [400]>

# Refresh Token

In [19]:
import requests
import base64

b64_id_secret = base64.b64encode(bytes(client_id + ':' + client_secret, 'utf-8')).decode('utf-8')


# open last refresh token
f = open('oath_refresh2.txt','r')
refresh_token = f.read()


# get new access token and refresh token
url = 'https://identity.xero.com/connect/token'
response = requests.post(url,
           headers={
               'Authorization': 'Basic ' + b64_id_secret
           },
           data={
               'grant_type': 'refresh_token',
               'client_id' : client_id,
               'refresh_token': refresh_token
           })


response.json()
response_dict = response.json()
access_token = response_dict['access_token']
refresh_token = response_dict['refresh_token']


# save new refresh token
f = open('oath_refresh.txt','w')
f = f.write(refresh_token)

# Check how many tenants in API

In [20]:
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
json.loads(response.text)
  

[{'id': 'dea2dd9c-9366-450e-8964-dd18e5814abc',
  'authEventId': '9dbcdca8-1f38-4caa-84fb-01870c08eed8',
  'tenantId': '6439291f-b6b3-433a-b6b3-810336a7c903',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Xero Testing2',
  'createdDateUtc': '2022-05-17T06:31:48.1219710',
  'updatedDateUtc': '2022-05-17T06:31:48.1231110'},
 {'id': '89e470f4-503a-44dc-af06-63f68e6769b2',
  'authEventId': '0cc2e136-620d-4a9a-80ba-c1ddb4887fb4',
  'tenantId': 'c2635a39-2e2a-4105-8aa9-b4800c4f59dc',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Xero Testing',
  'createdDateUtc': '2022-05-17T06:31:12.4294250',
  'updatedDateUtc': '2022-05-17T06:31:12.4306880'},
 {'id': 'aa7e2f9a-1077-4b18-9dda-04e37c9d0004',
  'authEventId': 'a3f5710e-c948-4110-95e1-3e8a9a9a7468',
  'tenantId': 'ef778bbb-e6cb-4f92-8cfe-4e165c0da7ee',
  'tenantType': 'ORGANISATION',
  'tenantName': 'Demo Company (AU)',
  'createdDateUtc': '2022-05-18T12:27:36.6094690',
  'updatedDateUtc': '2022-05-18T12:27:36.6111980'}]

# Get Trial Balance from Xero for a tenant

In [25]:
# put parameter for month
mymonth = '2022-05-31'
url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance?date='+mymonth+'&paymentsOnly=false'

In [26]:

#url = 'https://api.xero.com/api.xro/2.0/Contacts'
#url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance'
#url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance?date=2022-04-30&paymentsOnly=false'
#url = 'https://api.xero.com/connections'


response = requests.get(url,
       headers={
           'Authorization' : 'Bearer '+access_token,
           'Accept' : 'application/json',
           'Xero-tenant-id' : 'ef778bbb-e6cb-4f92-8cfe-4e165c0da7ee'})
data = json.loads(response.text)

# Convert Json result to dataframe

In [71]:

# get Clinic Name from Json API
for i in data['Reports']:
    for j in i['ReportTitles']:
        if 'Trial' not in j and 'As at' not in j:
            clinicname = j

# get Values from JSON API
mycell = []
for i in data['Reports']:
    for j, l1 in enumerate(i['Rows']):
        # only get the data from second row onwards, ignore headers 
        if j != 0:
            
            # grab section
            mysection = l1['Title']
            
            # go to dictionary rows section and loop it to find cells
            # from cells you can get all the values for each row
            for l2 in l1['Rows']:
                #print (l2,'\n')
                myrow = []
                myrow.append(mysection)
                for l3 in l2['Cells']:
                  #print (l3['Value'])
                  myrow.append(l3['Value'])
                mycell.append(myrow)

# create draft dataframe                
df = pd.DataFrame(mycell)
df.columns = ['Section','Desc','Dr','Cr','YTD Dr','YTD Cr']

# add additional information to dataframe
df['ClinicName'] = clinicname
df['Month'] = mymonth
df['AccountCode'] = df['Desc'].apply(lambda x:x[-4:-1])
df['Desc'] = df['Desc'].apply(lambda x:x[:-6])

# final dataframe
df = df[['Month','ClinicName','Section','AccountCode','Desc','Dr','Cr','YTD Dr','YTD Cr']]
df.columns = ['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit']

df

,Date,XeroOrganisation,SectionName,AccountCode,AccountName,Debit,Credit,YTD Debit,YTD Credit
0,2022-05-31,Demo Company (AU),Revenue,270,Interest Income,,97.05,,97.05
1,2022-05-31,Demo Company (AU),Revenue,200,Sales,,17447.32,,53378.32
2,2022-05-31,Demo Company (AU),Expenses,400,Advertising,2272.73,,4102.91,
3,2022-05-31,Demo Company (AU),Expenses,404,Bank Fees,300.00,,331.50,
4,2022-05-31,Demo Company (AU),Expenses,408,Cleaning,155.00,,465.00,
5,2022-05-31,Demo Company (AU),Expenses,412,Consulting & Accounting,49.00,,98.00,
6,2022-05-31,Demo Company (AU),Expenses,420,Entertainment,0.00,,277.20,
7,2022-05-31,Demo Company (AU),Expenses,425,Freight & Courier,,9.09,,9.09
8,2022-05-31,Demo Company (AU),Expenses,429,General Expenses,118.18,,1085.05,
9,2022-05-31,Demo Company (AU),Expenses,441,Legal expenses,4090.91,,4090.91,


# Do testing below this line

In [99]:
final = pd.DataFrame(columns=['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit'])

############ get list of tenants ################
url = 'https://api.xero.com/connections'
response = requests.get(url,
           headers={
               'Authorization' : 'Bearer '+access_token,
               'Accept' : 'application/json'})
myjson = json.loads(response.text)

mytenants = []
for i in myjson:
    mytenants.append(i['tenantId'])
###################################################


############ put parameter for month ##############
########## Change URL for TRIAL BALANCE ###########

mymonth = '2022-05-31'
url = 'https://api.xero.com/api.xro/2.0/Reports/TrialBalance?date='+mymonth+'&paymentsOnly=false'

###################################################


######### Get JSON API for each tenant ############
for mytenant in mytenants:
    response = requests.get(url,
            headers={
           'Authorization' : 'Bearer '+access_token,
           'Accept' : 'application/json',
           'Xero-tenant-id' : mytenant})
    data = json.loads(response.text)
###################################################



#### Modified the Json to DataFrame and CSV #######
    for i in data['Reports']:
        for j in i['ReportTitles']:
            if 'Trial' not in j and 'As at' not in j:
                clinicname = j

    # get Values from JSON API
    mycell = []
    for i in data['Reports']:
        for j, l1 in enumerate(i['Rows']):
            # only get the data from second row onwards, ignore headers 
            if j != 0:

                # grab section
                mysection = l1['Title']

                # go to dictionary rows section and loop it to find cells
                # from cells you can get all the values for each row
                for l2 in l1['Rows']:
                    #print (l2,'\n')
                    myrow = []
                    myrow.append(mysection)
                    for l3 in l2['Cells']:
                      #print (l3['Value'])
                      myrow.append(l3['Value'])
                    mycell.append(myrow)

    # create draft dataframe                
    df = pd.DataFrame(mycell)
    df.columns = ['Section','Desc','Dr','Cr','YTD Dr','YTD Cr']

    # add additional information to dataframe
    df['ClinicName'] = clinicname
    df['Month'] = mymonth
    df['AccountCode'] = df['Desc'].apply(lambda x:x[-4:-1])
    df['Desc'] = df['Desc'].apply(lambda x:x[:-6])

    # final dataframe
    df = df[['Month','ClinicName','Section','AccountCode','Desc','Dr','Cr','YTD Dr','YTD Cr']]
    df.columns = ['Date','XeroOrganisation','SectionName','AccountCode','AccountName','Debit','Credit','YTD Debit','YTD Credit']

    df.to_csv(clinicname+'.csv',index=False)
    final = final.append(df)
######################################################   

In [100]:
final

,Date,XeroOrganisation,SectionName,AccountCode,AccountName,Debit,Credit,YTD Debit,YTD Credit
0,2022-05-31,Xero Testing2,,ota,,0.00,0.00,0.00,0.00
0,2022-05-31,Xero Testing,Revenue,200,Sales,,3200.00,,3200.00
1,2022-05-31,Xero Testing,Assets,610,Accounts Receivable,3520.00,,3520.00,
2,2022-05-31,Xero Testing,Liabilities,820,GST,,320.00,,320.00
3,2022-05-31,Xero Testing,,ota,,3520.00,3520.00,3520.00,3520.00
0,2022-05-31,Demo Company (AU),Revenue,270,Interest Income,,97.05,,97.05
1,2022-05-31,Demo Company (AU),Revenue,200,Sales,,17447.32,,53378.32
2,2022-05-31,Demo Company (AU),Expenses,400,Advertising,2272.73,,4102.91,
3,2022-05-31,Demo Company (AU),Expenses,404,Bank Fees,300.00,,331.50,
4,2022-05-31,Demo Company (AU),Expenses,408,Cleaning,155.00,,465.00,
